# Imports and settings

In [1]:
from openmm.app import *
from openmm import *                    
from openmm.unit import *
from openmmtools import forces
import mdtraj as md
import numpy as np
import parmed as pmd
import bz2
import os
from openmm import CustomIntegrator
from openmm.unit import kilojoules_per_mole, is_quantity
from openmm.unit import *
import numpy as np
import pandas as pd 
from matplotlib import pyplot as plt

In [2]:
# define a function for creating RMSD restraints 
def create_rmsd_restraint(positions, atom_indicies):
    rmsd_cv = RMSDForce(positions, atom_indicies)
    energy_expression = 'step(dRMSD) * (K_RMSD/2) * dRMSD^2; dRMSD = (RMSD-RMSD0);'                                                  
    energy_expression += 'K_RMSD = %f;' % spring_constant.value_in_unit_system(md_unit_system)                                       
    energy_expression += 'RMSD0 = %f;' % restraint_distance.value_in_unit_system(md_unit_system)                                     
    restraint_force = CustomCVForce(energy_expression)                                                                               
    restraint_force.addCollectiveVariable('RMSD', rmsd_cv)                                                                           
    return restraint_force                                                                                                           
                                                                  

In [3]:
# define a function to list force groups and modify system context                                                       
def forcegroupify(system):                                        
    forcegroups = {}                                              
    for i in range(system.getNumForces()):                                                                                           
        force = system.getForce(i)                           
        force.setForceGroup(i) 
        forcegroups[force] = i                                    
    return forcegroups

# Create system + CV Forces with harmonic restraints

In [4]:
sim_temp = 300.0 * kelvin
H_mass = 4.0 * amu #Might need to be tuned to 3.5 amu 
time_step = 0.002 * picosecond  
nb_cutoff = 10.0 * angstrom                                                                                                          
box_padding = 12.0 * angstrom
salt_conc = 0.15 * molar
receptor_path="../villin.pdb"
current_file="villin-solvated"
# Misc parameters                                                 
restraint_distance = 0.0 * angstroms                              
restart_freq = 10
log_freq = 1                                                                                                                         
prd_steps = 100     

In [5]:
## Load an already solvated PDB file and set up the system + state
pdb = PDBFile(receptor_path)
omm_forcefield = ForceField("amber/ff14SB.xml", "amber14/tip3p.xml")
system = omm_forcefield.createSystem(pdb.topology, 
                                         nonbondedMethod=PME, 
                                         nonbondedCutoff=nb_cutoff, 
                                         constraints=HBonds, 
                                         rigidWater=True,
                                         hydrogenMass=H_mass)
system.addForce(MonteCarloBarostat(1*bar, sim_temp))


5

In [6]:
prot_top = md.Topology.from_openmm(pdb.topology)
# find three atoms and save their inds - these will be used to define distances
d1_atom1_ind = prot_top.select("residue 6 and name CZ")[0]
d1_atom2_ind = prot_top.select("residue 10 and name CZ")[0] 
d2_atom2_ind = prot_top.select("residue 17 and name CZ")[0]

In [7]:
# Add Harmonica functions to CV bonds
bias_d1 = CustomBondForce("0.5*k*(r-r0_d1)^2")
bias_d1.addGlobalParameter("k", 1.0)
bias_d1.addGlobalParameter("r0_d1", 0.3)

bias_d2 = CustomBondForce("0.5*k*(r-r0_d2)^2")
bias_d2.addGlobalParameter("k", 1.0)
bias_d2.addGlobalParameter("r0_d2", 0.3)

1

In [8]:
# Add forces to the system
bias_d1.addBond(d1_atom1_ind, d1_atom2_ind)
bias_d2.addBond(d1_atom1_ind, d2_atom2_ind)

system.addForce(bias_d1)
system.addForce(bias_d2)

7

# Define an integrator where the harmonic boost potential applied is written out as a global variable

In [9]:
import numpy as np
kB = BOLTZMANN_CONSTANT_kB * AVOGADRO_CONSTANT_NA

'''
Useful links in the writing of this integrator: 
Writing custom integrators: https://github.com/choderalab/openmm-tutorials/blob/master/02%20-%20Integrators%20and%20sampling.ipynb
AMD integrator: https://github.com/openmm/openmm/blob/master/wrappers/python/openmm/amd.py
'''

class LangevinEqIntegrator(CustomIntegrator):
    """LangevinEqIntegrator is a standard Langevin Eq. Integrator but it stores the energy
    as a stored GlobalParameter for easy reporting via FAH
    """

    def __init__(self, dt, temperature=300*kelvin, collision_rate=1.0/picosecond):
        """Create an LangevinEqIntegrator.
        Parameters
        ----------
        dt : time 
            The integration time step to use (simtk quantity)
        temperature : temperature
            temperature of the system
        collision rate : collision rate
            Collision for the thermostat to update on
        """
        gamma = collision_rate*picoseconds # add this because SWIG somehow?
        CustomIntegrator.__init__(self, dt)
        # parameters
        self.addPerDofVariable("oldx", 0)
        self.addGlobalVariable("V0", 0)
        self.addPerDofVariable("sigma", 0)
        self.addUpdateContextState();
        
        # Comment out old AMD method 
        # self.addComputePerDof("v", "v+dt*fprime/m; fprime=f*((1-modify) + modify*(alpha/(alpha+E-energy))^2); modify=step(E-energy)")
        # self.addComputePerDof("oldx", "x")
        # self.addComputePerDof("x", "x+dt*v")
        # self.addConstrainPositions()
        # self.addComputePerDof("v", "(x-oldx)/dt")

        # new variables for Langevin kernel
        self.addGlobalVariable('kT', kB * temperature)
        self.addComputePerDof("sigma", "sqrt(kT/m)")
        self.addGlobalVariable("a", np.exp(-1 * gamma)) #vscale?
        self.addGlobalVariable("b", np.sqrt(1 - np.exp(-2 * gamma))) # noise-scale?

        #before position restraints
        #self.addPerDofVariable("oldx", 0)

        # Compute steps for Langevin - attempt at VRORV splitting?
        self.addComputePerDof("v", "v + (dt / 2) * f/m")
        # langevin like normal
        self.addComputePerDof("x", "x + (dt / 2)*v")
        self.addComputePerDof("oldx", "x")
        self.addConstrainPositions()
        self.addComputePerDof("v", "v + (x - oldx)/(dt / 2)")
        self.addComputePerDof("v", "(a * v) + (b * sigma * gaussian)") # taken from openmm-tutorial on custom integrators (link 1 above)
        self.addComputePerDof("x", "x + (dt / 2)*v")
        self.addComputePerDof("oldx", "x")
        self.addConstrainPositions()
        self.addComputePerDof("v", "v + (x - oldx) / (dt / 2)")
        # Use the same line as what you calculated above
        self.addComputePerDof("v", "v + (dt / 2) * f/m")
        # compute the V and deltaV values
        self.addComputeGlobal("V0", "energy")
        self.addConstrainVelocities()


In [10]:
integrator = LangevinEqIntegrator(dt=time_step,
                                  temperature=300*kelvin,
                                  collision_rate=1.0/picosecond)

# Create the simulation object

In [12]:
platform = Platform.getPlatformByName('OpenCL')
platform.setPropertyDefaultValue('Precision', 'mixed')

In [13]:
simulation = Simulation(pdb.topology,system, integrator, platform)

In [14]:
simulation.reporters.append(DCDReporter('./'+current_file+''+ '.dcd', restart_freq))
simulation.reporters.append(CheckpointReporter('./'+current_file+''+ '.chk', min(prd_steps, 10*restart_freq)))
simulation.reporters.append(StateDataReporter(open('./log.' + current_file+'', 'w'), log_freq, step=True, potentialEnergy=True, kineticEnergy=True, totalEnergy=True, temperature=True, volume=True, density=True, speed=True))

# Set up a single simulation with one umbrella (in reality you'd do multiple of these)


In [15]:
## set force constant K for the biasing potential.
## the unit here is kJ*mol^{-1}*nm^{-2}, which is the default unit used in OpenMM
K = 100
simulation.context.setParameter("k", K)

## M centers of harmonic biasing potentials
M = 20
r0_range = np.linspace(0.3, 2.0, M, endpoint = False)

In [16]:
simulation.context.setParameter('r0_d1', r0_range[1])
simulation.context.setParameter('r0_d2', r0_range[2])

In [18]:
simulation.context.setPositions(pdb.positions)
print('  initial : %s' % (simulation.context.getState(getEnergy=True).getPotentialEnergy()))
simulation.minimizeEnergy()
print('  final : %s' % (simulation.context.getState(getEnergy=True).getPotentialEnergy()))

  initial : -115004.21780371421 kJ/mol
  final : -145473.62098196533 kJ/mol


In [19]:
for i in range(5):
    simulation.step(10)

If we were to do this for multiple windows, the way to do this is set up a loop over the windows, and then run the simulation for each window.\
For each system, you'd just have to first define a set of umbrellas with something like:

```
M = 20
r0_range = np.linspace(0.3, 2.0, M, endpoint = False)
```
Then for each new system (1 per window), you'd add it like: 
```
simulation.context.setParameter('r0_d1', r0_range[m])
```